In [1]:
!pip install crewai python-dotenv PyPDF2 python-docx PyMuPDF langchain langchain-google-genai google-generativeai beautifulsoup4 requests opencv-python numpy pytesseract PyMuPDF Pillow pyzbar 



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv
import json
import re
import pandas as pd
from typing import List, Dict, Any, Optional

import traceback
from urllib.parse import quote_plus
import PyPDF2
import docx
import fitz  # PyMuPDF
import base64
from io import BytesIO

# For web scraping
import requests
from bs4 import BeautifulSoup

# For LLM integrations
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI as GoogleGenerativeAI
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatLiteLLM

# For Crew AI
from crewai import Agent, Task, Process, Crew
from crewai.tools.base_tool import BaseTool as Tool


import cv2
import io
import numpy as np
import pytesseract
import re
import fitz  # PyMuPDF
from PIL import Image
from pyzbar.pyzbar import decode
from urllib.parse import urlparse

c:\Users\knand\OneDrive\Desktop\skill ass\skill_ass\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load environment variables
load_dotenv()

# Configure API keys
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Verify API keys are loaded
print(f"SERPER_API_KEY loaded: {'Yes' if SERPER_API_KEY else 'No'}")
print(f"GEMINI_API_KEY loaded: {'Yes' if GEMINI_API_KEY else 'No'}")


SERPER_API_KEY loaded: Yes
GEMINI_API_KEY loaded: Yes


In [4]:
# Configure Gemini API
genai.configure(api_key=GEMINI_API_KEY)
gemini_pro = genai.GenerativeModel('gemini-1.5-flash')
print("Environment and API configurations loaded successfully.")


Environment and API configurations loaded successfully.


In [5]:
class SearchTool(Tool):
    name: str = "Search Tool"
    description: str = "Useful for searching information about technologies, companies, or people online."
    
    def _run(self, query: str) -> str:  # Added self parameter
        try:
            url = "https://google.serper.dev/search"
            payload = json.dumps({"q": query})
            headers = {
                'X-API-KEY': SERPER_API_KEY,
                'Content-Type': 'application/json'
            }
            response = requests.request("POST", url, headers=headers, data=payload)
            return response.text
        except Exception as e:
            return f"Error during search: {str(e)}"

In [6]:
# 4.2 GitHub Profile Scraper
class GitHubProfileScraper(Tool):
    name: str = "GitHub Profile Scraper"
    description: str = "Scrapes a GitHub profile to gather information about repositories, contributions, and skills."
    
    def _run(self, github_username: str) -> str:  # Added self parameter
        try:
            # Rest of the method stays the same
            profile_url = f"https://api.github.com/users/{github_username}"
            repos_url = f"https://api.github.com/users/{github_username}/repos"
            
            # Get profile information
            profile_response = requests.get(profile_url)

            if profile_response.status_code != 200:
                return f"Error: Could not retrieve GitHub profile for {github_username}."
            
            profile_data = profile_response.json()
            
            # Get repositories information
            repos_response = requests.get(repos_url)
            if repos_response.status_code != 200:
                return f"Error: Could not retrieve repositories for {github_username}."
            
            repos_data = repos_response.json()
            
            # Extract relevant information
            total_repos = len(repos_data)
            languages_used = {}
            starred_repos = 0
            forks = 0
            
            for repo in repos_data:
                forks += repo['forks_count']
                starred_repos += repo['stargazers_count']
                
                # Get languages used in each repository
                if not repo['fork']:  # Only consider non-forked repos
                    lang_url = repo['languages_url']
                    lang_response = requests.get(lang_url)
                    if lang_response.status_code == 200:
                        langs = lang_response.json()
                        for lang, bytes_of_code in langs.items():
                            if lang in languages_used:
                                languages_used[lang] += bytes_of_code
                            else:
                                languages_used[lang] = bytes_of_code
            
            # Sort languages by bytes of code
            sorted_languages = sorted(languages_used.items(), key=lambda x: x[1], reverse=True)
            top_languages = [lang for lang, _ in sorted_languages[:5]]
            
            # Format the results
            result = {
                "username": github_username,
                "name": profile_data.get('name', 'Not available'),
                "bio": profile_data.get('bio', 'Not available'),
                "public_repos": total_repos,
                "followers": profile_data.get('followers', 0),
                "following": profile_data.get('following', 0),
                "top_languages": top_languages,
                "starred_repos_count": starred_repos,
                "forks_count": forks,
                "profile_created_at": profile_data.get('created_at', 'Not available')
            }
            
            return json.dumps(result, indent=2)
        except Exception as e:
            return f"Error scraping GitHub profile: {str(e)}\n{traceback.format_exc()}"


In [7]:
# 4.3 LeetCode Profile Scraper
class LeetCodeProfileScraper(Tool):
    name: str = "LeetCode Profile Scraper"
    description: str = "Scrapes a LeetCode profile to gather information about solved problems and contest ratings."
    
    def _run(self, leetcode_username: str) -> str:  # Added self parameter
        try:
            # Using public GraphQL API for LeetCode
            url = "https://leetcode.com/graphql"
            
            # Query to get user profile information
            query = """
            query userPublicProfile($username: String!) {
              matchedUser(username: $username) {
                username
                submitStats: submitStatsGlobal {
                  acSubmissionNum {
                    difficulty
                    count
                    submissions
                  }
                }
                profile {
                  ranking
                  reputation
                  starRating
                }
                badges {
                  id
                  name
                  icon
                }
              }
            }
            """
            
            variables = {"username": leetcode_username}
            payload = {"query": query, "variables": variables}
            headers = {
                "Content-Type": "application/json",
                "Referer": f"https://leetcode.com/{leetcode_username}/"
            }
            
            response = requests.post(url, headers=headers, json=payload)
            
            if response.status_code != 200:
                return f"Error: Could not retrieve LeetCode profile for {leetcode_username}."
            
            data = response.json()
            
            if not data.get('data', {}).get('matchedUser'):
                return f"Error: LeetCode user {leetcode_username} not found."
            
            user_data = data['data']['matchedUser']
            
            # Extract relevant information
            submission_stats = user_data['submitStats']['acSubmissionNum']
            problem_counts = {}
            
            for stat in submission_stats:
                problem_counts[stat['difficulty']] = stat['count']
            
            total_solved = sum(problem_counts.values())
            
            # Format the results
            result = {
                "username": leetcode_username,
                "total_problems_solved": total_solved,
                "problems_by_difficulty": problem_counts,
                "ranking": user_data['profile'].get('ranking', 'Not available'),
                "reputation": user_data['profile'].get('reputation', 'Not available'),
                "badges": [badge['name'] for badge in user_data.get('badges', [])]
            }
            
            return json.dumps(result, indent=2)
        except Exception as e:
            return f"Error scraping LeetCode profile: {str(e)}\n{traceback.format_exc()}"


In [8]:

class CertificateVerificationTool(Tool):
    name: str = "Certificate Verification Tool"
    description: str = "Verifies certificates by analyzing certificate URLs, extracting text via OCR, detecting QR codes, and validating candidate credentials."
    
    def _run(self, certificate_url: str) -> str:
        """Verifies a certificate by analyzing its URL and content."""
        try:
            # Check if the URL is valid
            if not self._is_valid_url(certificate_url):
                return json.dumps({
                    "status": "failed",
                    "reason": "Invalid URL format",
                    "url": certificate_url
                })
                
            # Download the certificate image or PDF
            content, content_type = self._download_content(certificate_url)
            if content is None:
                return json.dumps({
                    "status": "failed",
                    "reason": "Could not download content",
                    "url": certificate_url
                })
            
            # Extract text and analyze certificate
            result = self._analyze_certificate(content, content_type, certificate_url)
            return json.dumps(result, indent=2)
            
        except Exception as e:
            return json.dumps({
                "status": "error",
                "message": str(e),
                "url": certificate_url
            })
    
    def _is_valid_url(self, url: str) -> bool:
        """Check if the URL is valid."""
        try:
            result = urlparse(url)
            return all([result.scheme, result.netloc])
        except:
            return False
    
    def _download_content(self, url: str):
        """Download content from URL and determine its type."""
        try:
            response = requests.get(url, stream=True)
            if response.status_code != 200:
                return None, None
                
            content_type = response.headers.get('Content-Type', '').lower()
            
            if 'application/pdf' in content_type:
                return response.content, 'pdf'
            elif any(img_type in content_type for img_type in ['image/jpeg', 'image/png', 'image/jpg']):
                return response.content, 'image'
            else:
                # Try to determine based on URL
                if url.lower().endswith(('.pdf')):
                    return response.content, 'pdf'
                elif url.lower().endswith(('.jpg', '.jpeg', '.png')):
                    return response.content, 'image'
                else:
                    # Default to binary
                    return response.content, 'unknown'
                    
        except Exception as e:
            print(f"Error downloading content: {str(e)}")
            return None, None
    
    def _analyze_certificate(self, content, content_type, url):
        """Analyze certificate content to extract information and verify it."""
        extracted_text = ""
        qr_data = []
        issuer = "Unknown"
        cert_name = "Unknown"
        
        # Extract text based on content type
        if content_type == 'pdf':
            extracted_text, qr_data = self._analyze_pdf(content)
        elif content_type == 'image':
            extracted_text, qr_data = self._analyze_image(content)
        else:
            return {
                "status": "failed",
                "reason": "Unsupported content type",
                "url": url
            }
        
        # Process the extracted text
        if extracted_text:
            # Extract certificate info
            cert_info = self._extract_certificate_info(extracted_text, url)
            cert_name = cert_info.get("certificate_name", "Unknown")
            issuer = cert_info.get("issuer", "Unknown")
            
            # Determine verification status
            verification_status = "verified" if cert_info.get("verified", False) else "unverified"
            
            return {
                "status": verification_status,
                "url": url,
                "certificate_name": cert_name,
                "issuer": issuer,
                "recipient_name": cert_info.get("recipient_name", "Not found"),
                "issue_date": cert_info.get("issue_date", "Not found"),
                "expiry_date": cert_info.get("expiry_date", "Not found"),
                "qr_links": qr_data,
                "skills": cert_info.get("skills", []),
                "confidence": cert_info.get("confidence", "low")
            }
        else:
            return {
                "status": "failed",
                "reason": "Could not extract text from certificate",
                "url": url
            }
    
    def _analyze_pdf(self, content):
        """Extract text and QR codes from PDF."""
        extracted_text = ""
        qr_data = []
        
        try:
            # Load PDF
            pdf_document = fitz.open(stream=content, filetype="pdf")
            
            # Process each page
            for page_num in range(len(pdf_document)):
                page = pdf_document[page_num]
                
                # Extract text
                page_text = page.get_text()
                extracted_text += page_text + "\n"
                
                # Extract images for QR detection
                image_list = page.get_images(full=True)
                for img_index, img in enumerate(image_list):
                    xref = img[0]
                    base_image = pdf_document.extract_image(xref)
                    image_bytes = base_image["image"]
                    
                    # Try to decode QR
                    try:
                        pil_image = Image.open(io.BytesIO(image_bytes))
                        codes = decode(pil_image)
                        for code in codes:
                            qr_data.append(code.data.decode('utf-8'))
                    except Exception as e:
                        print(f"Error decoding QR from PDF image: {str(e)}")
            
            return extracted_text, qr_data
            
        except Exception as e:
            print(f"Error analyzing PDF: {str(e)}")
            return "", []
    
    def _analyze_image(self, content):
        """Extract text and QR codes from image."""
        extracted_text = ""
        qr_data = []
        
        try:
            # Convert to numpy array for OpenCV
            nparr = np.frombuffer(content, np.uint8)
            image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            
            # Convert to grayscale for better OCR
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            # Apply pre-processing for better OCR
            thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
            
            # Extract text with PyTesseract
            extracted_text = pytesseract.image_to_string(thresh)
            
            # Detect QR codes
            pil_image = Image.open(io.BytesIO(content))
            codes = decode(pil_image)
            for code in codes:
                qr_data.append(code.data.decode('utf-8'))
                
            return extracted_text, qr_data
            
        except Exception as e:
            print(f"Error analyzing image: {str(e)}")
            return "", []
    
    def _extract_certificate_info(self, text, url):
        """Extract certificate information from text."""
        info = {
            "certificate_name": "Unknown",
            "issuer": "Unknown",
            "recipient_name": "Not found",
            "issue_date": "Not found",
            "expiry_date": "Not found",
            "verified": False,
            "skills": [],
            "confidence": "low"
        }
        
        # Common certificate issuers
        issuers = [
            "Coursera", "Udemy", "edX", "LinkedIn Learning", "Microsoft", "IBM", 
            "Google", "Amazon AWS", "Oracle", "Udacity", "Pluralsight", 
            "FreeCodeCamp", "DataCamp", "HackerRank", "SoloLearn"
        ]
        
        # Find issuer
        for issuer in issuers:
            if re.search(issuer, text, re.IGNORECASE):
                info["issuer"] = issuer
                break
                
        # Try to determine if the URL indicates the issuer
        domain = urlparse(url).netloc.lower()
        for issuer in issuers:
            if issuer.lower().replace(" ", "") in domain:
                info["issuer"] = issuer
                break
        
        # Find certificate name
        cert_patterns = [
            r"certificate (?:of|in|for) ([^\n.]+)",
            r"certified ([^\n.]+) (?:specialist|professional|developer|architect)",
            r"([^\n.]+) certification",
            r"([^\n.]+) certificate"
        ]
        
        for pattern in cert_patterns:
            matches = re.search(pattern, text, re.IGNORECASE)
            if matches:
                info["certificate_name"] = matches.group(1).strip()
                break
        
        # Extract recipient name (common patterns in certificates)
        name_patterns = [
            r"(?:presented to|awarded to|issued to|certifies that|this is to certify that|granted to) ([A-Z][a-z]+ [A-Z][a-z]+)",
            r"(?:name|participant|recipient)[:\s]+([A-Z][a-z]+ [A-Z][a-z]+)",
            r"([A-Z][a-z]+ [A-Z][a-z]+) has (?:successfully completed|achieved|earned|passed)"
        ]
        
        for pattern in name_patterns:
            matches = re.search(pattern, text, re.IGNORECASE)
            if matches:
                info["recipient_name"] = matches.group(1).strip()
                break
        
        # Extract dates
        date_patterns = [
            r"(?:issued|dated|date|awarded on)[:\s]+(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})",
            r"(?:issued|dated|date|awarded on)[:\s]+(\w+ \d{1,2},? \d{4})",
            r"(\d{1,2} \w+ \d{4})"
        ]
        
        expiry_patterns = [
            r"(?:valid until|expires on|expiry date|valid through)[:\s]+(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})",
            r"(?:valid until|expires on|expiry date|valid through)[:\s]+(\w+ \d{1,2},? \d{4})",
        ]
        
        # Find issue date
        for pattern in date_patterns:
            matches = re.search(pattern, text, re.IGNORECASE)
            if matches:
                info["issue_date"] = matches.group(1).strip()
                break
        
        # Find expiry date if available
        for pattern in expiry_patterns:
            matches = re.search(pattern, text, re.IGNORECASE)
            if matches:
                info["expiry_date"] = matches.group(1).strip()
                break
        
        # Common skill keywords associated with certificates
        skill_keywords = {
            "Programming": ["Python", "Java", "JavaScript", "C++", "Ruby", "PHP", "C#", "Swift", "Go"],
            "Web Development": ["HTML", "CSS", "React", "Angular", "Vue", "Node.js", "Django", "Flask", "Express", "Laravel", "Spring"],
            "Data": ["SQL", "MySQL", "PostgreSQL", "MongoDB", "NoSQL", "Data Science", "Machine Learning", "AI", "Deep Learning", "NLP", "Computer Vision"],
            "Cloud": ["AWS", "Azure", "Google Cloud", "GCP", "Docker", "Kubernetes", "Terraform", "DevOps", "CI/CD", "Lambda"],
            "Other": ["Project Management", "Agile", "Scrum", "Security", "Ethical Hacking", "Network", "Linux", "Big Data", "Hadoop", "Spark"]
        }
        
        # Extract skills from certificate text
        extracted_skills = []
        for category, skills in skill_keywords.items():
            for skill in skills:
                if re.search(r'\b' + re.escape(skill) + r'\b', text, re.IGNORECASE):
                    extracted_skills.append(skill)
        
        info["skills"] = extracted_skills
        
        # Verify if a recipient name was found
        if info["recipient_name"] != "Not found":
            info["verified"] = True
            info["confidence"] = "medium"
            
            # Increase confidence if multiple pieces of information are found
            confidence_score = 0
            if info["certificate_name"] != "Unknown":
                confidence_score += 1
            if info["issuer"] != "Unknown":
                confidence_score += 1
            if info["issue_date"] != "Not found":
                confidence_score += 1
            if len(info["skills"]) > 0:
                confidence_score += 1
                
            if confidence_score >= 3:
                info["confidence"] = "high"
        
        return info

class SkillMappingTool(Tool):
    name: str = "Skill Mapping Tool"
    description: str = "Maps certifications to relevant technical skills with proficiency levels."
    
    def _run(self, certification_data: str) -> str:
        """Maps certifications to skills with estimated proficiency levels."""
        try:
            certifications = json.loads(certification_data)
            
            skill_mappings = self._map_certifications_to_skills(certifications)
            
            return json.dumps({"skill_mappings": skill_mappings}, indent=2)
            
        except Exception as e:
            return json.dumps({
                "status": "error",
                "message": str(e)
            })
    
    def _map_certifications_to_skills(self, certifications):
        """Map certifications to skills with estimated proficiency levels."""
        skill_mappings = {}
        
        # Certification level to skill proficiency mappings
        certification_levels = {
            "basic": {"level": 3, "description": "Foundational knowledge"},
            "beginner": {"level": 3, "description": "Foundational knowledge"},
            "intermediate": {"level": 5, "description": "Working proficiency"},
            "advanced": {"level": 7, "description": "Advanced proficiency"},
            "expert": {"level": 9, "description": "Expert level knowledge"}
        }
        
        # Common certification issuers and their typical quality score (1-10)
        issuer_quality = {
            "Microsoft": 8,
            "AWS": 8,
            "Google": 8,
            "Oracle": 8,
            "Cisco": 8,
            "CompTIA": 7,
            "IBM": 7,
            "Salesforce": 7,
            "Adobe": 7,
            "PMI": 8,
            "Coursera": 6,
            "Udemy": 5,
            "edX": 6,
            "LinkedIn Learning": 5,
            "Udacity": 6,
            "Pluralsight": 5,
            "HackerRank": 6,
            "FreeCodeCamp": 5,
            "DataCamp": 6,
            "SoloLearn": 4
        }
        
        # Common certification prefixes that indicate level
        level_indicators = {
            "fundamentals": "basic",
            "foundation": "basic",
            "associate": "intermediate",
            "professional": "advanced",
            "expert": "expert",
            "master": "expert",
            "specialization": "advanced",
            "specialist": "intermediate",
            "practitioner": "intermediate",
            "advanced": "advanced",
            "beginner": "basic",
            "introduction": "basic"
        }
        
        # Certification to skills mapping
        cert_skill_mappings = {
            # AWS
            "AWS Certified Solutions Architect": ["AWS", "Cloud Architecture", "EC2", "S3", "IAM", "VPC", "RDS"],
            "AWS Certified Developer": ["AWS", "Lambda", "DynamoDB", "API Gateway", "CloudFormation", "CI/CD"],
            "AWS Certified DevOps Engineer": ["AWS", "DevOps", "CloudFormation", "CI/CD", "Docker", "Monitoring"],
            
            # Microsoft
            "Microsoft Certified: Azure": ["Azure", "Cloud Computing"],
            "Microsoft Certified: Data": ["SQL Server", "Data Engineering", "Data Analytics"],
            "Microsoft Certified: Azure Solutions Architect": ["Azure", "Cloud Architecture", "Azure Resources", "Security"],
            "Microsoft Certified: Azure Developer": ["Azure", ".NET", "Azure Functions", "Azure App Service", "Cosmos DB"],
            
            # Google
            "Google Cloud Certified": ["Google Cloud", "Cloud Computing"],
            "Google Associate Cloud Engineer": ["Google Cloud", "GCE", "GKE", "Cloud Storage", "IAM"],
            "Google Professional Cloud Architect": ["Google Cloud", "Cloud Architecture", "GCP Solutions", "Security"],
            "Google Professional Data Engineer": ["BigQuery", "Dataflow", "Machine Learning", "Data Processing"],
            
            # Programming
            "Python": ["Python", "Programming"],
            "Java": ["Java", "Programming", "OOP"],
            "JavaScript": ["JavaScript", "Web Development", "Frontend"],
            "React": ["React", "Frontend", "JavaScript", "Web Development"],
            "Angular": ["Angular", "Frontend", "TypeScript", "Web Development"],
            "Node.js": ["Node.js", "JavaScript", "Backend", "Web Development"],
            "Full Stack": ["Frontend", "Backend", "Database", "Web Development"],
            
            # Data Science
            "Data Science": ["Python", "Statistics", "Machine Learning", "Data Analysis"],
            "Machine Learning": ["Python", "Machine Learning", "Data Modeling", "Algorithms"],
            "Deep Learning": ["Neural Networks", "TensorFlow", "PyTorch", "Computer Vision", "NLP"],
            "TensorFlow": ["TensorFlow", "Machine Learning", "Neural Networks"],
            
            # Misc
            "Project Management": ["Project Management", "Agile", "Scrum"],
            "Scrum": ["Agile", "Scrum", "Project Management"],
            "Kubernetes": ["Kubernetes", "Container Orchestration", "Docker", "DevOps"],
            "Docker": ["Docker", "Containers", "DevOps"],
            "Cybersecurity": ["Security", "Network Security", "Ethical Hacking", "Threat Analysis"]
        }
        
        for cert in certifications:
            cert_name = cert.get("certificate_name", "").strip()
            if not cert_name or cert_name == "Unknown":
                continue
                
            issuer = cert.get("issuer", "Unknown").strip()
            status = cert.get("status", "unverified").lower()
            confidence = cert.get("confidence", "low").lower()
            skills = cert.get("skills", [])
            
            # Skip unverified certificates
            if status != "verified":
                continue
                
            # Determine certificate level
            cert_level = "intermediate"  # Default level
            for indicator, level in level_indicators.items():
                if indicator.lower() in cert_name.lower():
                    cert_level = level
                    break
                    
            # Base proficiency level from certificate level
            base_proficiency = certification_levels.get(cert_level, {"level": 5})["level"]
            
            # Adjust by issuer quality
            issuer_bonus = issuer_quality.get(issuer, 5) / 10
            
            # Adjust by confidence level
            confidence_multiplier = {
                "high": 1.0,
                "medium": 0.8,
                "low": 0.6
            }.get(confidence, 0.7)
            
            # Find matching skills based on certificate name
            matched_skills = []
            
            # First try exact matches
            for cert_pattern, related_skills in cert_skill_mappings.items():
                if cert_pattern.lower() in cert_name.lower():
                    matched_skills.extend(related_skills)
            
            # Add skills explicitly mentioned in the certificate
            matched_skills.extend(skills)
            
            # Remove duplicates
            matched_skills = list(set(matched_skills))
            
            # Calculate proficiency for each skill
            for skill in matched_skills:
                proficiency = round(base_proficiency * issuer_bonus * confidence_multiplier)
                proficiency = max(1, min(10, proficiency))  # Ensure between 1-10
                
                if skill in skill_mappings:
                    # Take the highest proficiency level if the skill appears in multiple certificates
                    skill_mappings[skill] = max(skill_mappings[skill], proficiency)
                else:
                    skill_mappings[skill] = proficiency
        
        return skill_mappings

In [9]:
def extract_text_from_pdf(pdf_file):
        """Extract text from a PDF file using multiple extraction methods for redundancy."""
        text = ""
        temp_file_path = None
        
        try:
            # First attempt: Use PyMuPDF (fitz) for better text extraction
            pdf_document = fitz.open(stream=pdf_file.read(), filetype="pdf")
            
            # Extract text page by page
            for page_num in range(len(pdf_document)):
                page = pdf_document[page_num]
                page_text = page.get_text()
                text += page_text
                
                # Extract links from annotations
                links = page.get_links()
                for link in links:
                    if 'uri' in link:
                        text += f"\nLink: {link['uri']}\n"
            
            pdf_document.close()
            
            # If text extraction yielded minimal results, try OCR fallback
            if len(text.strip()) < 100:
                print("Minimal text extracted. PDF might be scanned or secured.")
                # Here you could implement OCR using pytesseract
                pass
                
        except Exception as e:
            print(f"Error extracting text with PyMuPDF: {str(e)}")
            
            # Fallback to PyPDF2
            try:
                pdf_file.seek(0)
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                for page in pdf_reader.pages:
                    page_text = page.extract_text() or ""
                    text += page_text
            except Exception as e2:
                print(f"Error with fallback PDF extraction: {str(e2)}")
                
        return text


In [10]:
def extract_text_from_docx(docx_file):
        
        """Extract text from a DOCX file including hyperlinks."""
        text = ""
        try:
            doc = docx.Document(docx_file)
            
            # Extract text from paragraphs
            for para in doc.paragraphs:
                text += para.text + "\n"
                
                # Extract hyperlinks from paragraph runs
                for run in para.runs:
                    if run.element.findall('.//w:hyperlink', {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                        for hyperlink in run.element.findall('.//w:hyperlink', {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                            for relationship_id in hyperlink.attrib.values():
                                # This is a simple approach - a more robust solution would look up the relationship ID
                                # in the document's relationships to get the actual URL
                                text += f"\nHyperlink: {relationship_id}\n"
            
            # Extract text from tables
            for table in doc.tables:
                for row in table.rows:
                    for cell in row.cells:
                        text += cell.text + " | "
                    text += "\n"
                    
        except Exception as e:
            print(f"Error extracting text from DOCX: {str(e)}")
            
        return text

In [11]:
def extract_text_from_document(file):
        """Extract text from various document formats with enhanced link extraction."""
        try:
            # Get file extension
            file_name = file.name.lower() if hasattr(file, 'name') else "unknown_file"
            
            if file_name.endswith('.pdf'):
                return extract_text_from_pdf(file)
            elif file_name.endswith('.docx'):
                return extract_text_from_docx(file)
            elif file_name.endswith('.txt'):
                return file.read().decode('utf-8')
            else:
                return f"Unsupported file format: {file_name.split('.')[-1]}"
                
        except Exception as e:
            print(f"Error processing document: {str(e)}")
            return f"Error processing document: {str(e)}"

In [12]:
def extract_name_from_resume(resume_data):
    """Extract candidate name from parsed resume data."""
    if isinstance(resume_data, dict):
        # Try to find name in personal_information section first
        if "personal_information" in resume_data and "name" in resume_data["personal_information"]:
            return resume_data["personal_information"]["name"]
        
        # Look for name in other common locations
        if "name" in resume_data:
            return resume_data["name"]
        if "basic_info" in resume_data and "name" in resume_data["basic_info"]:
            return resume_data["basic_info"]["name"]
        
        # Try to find any keys that might contain name
        for key in resume_data:
            if isinstance(resume_data[key], dict) and "name" in resume_data[key]:
                return resume_data[key]["name"]
    
    # Default if we can't find a specific name
    return "Candidate"

In [13]:
import re
import json

def extract_profile_links(text):
    """Extract coding profile links and other professional profile links from text."""
    profiles = {}
    
    # GitHub profile patterns - enhanced with multiple formats
    github_patterns = [
        r'github\.com/([a-zA-Z0-9_-]+)',
        r'github:?\s*:?\s*([a-zA-Z0-9_-]+)',
        r'GitHub\s*Profile:?\s*([a-zA-Z0-9_-]+)',
        r'[Gg][Ii][Tt][Hh][Uu][Bb][:\s]+\s*([a-zA-Z0-9_-]+)',
        r'[Gg][Ii][Tt][Hh][Uu][Bb].*?[:/]([a-zA-Z0-9_-]+)',
    ]
    
    # LeetCode profile patterns - enhanced with multiple formats
    leetcode_patterns = [
        r'leetcode\.com/([a-zA-Z0-9_-]+)',
        r'leetcode:?\s*:?\s*([a-zA-Z0-9_-]+)',
        r'LeetCode\s*Profile:?\s*([a-zA-Z0-9_-]+)',
        r'[Ll][Ee][Ee][Tt][Cc][Oo][Dd][Ee][:\s]+\s*([a-zA-Z0-9_-]+)',
        r'[Ll][Ee][Ee][Tt][Cc][Oo][Dd][Ee].*?[:/]([a-zA-Z0-9_-]+)',
    ]
    
    # Certification link patterns
    certification_patterns = [
        r'certification:?\s*[^\n]*?(https?://[^\s]+)',
        r'certificate:?\s*[^\n]*?(https?://[^\s]+)',
        r'(https?://[^\s]*certificate[^\s]*)',
        r'(https?://[^\s]*certify[^\s]*)',
        r'(https?://[^\s]*verify[^\s]*)',
        r'(https?://[^\s]*credential[^\s]*)',
    ]
    
    # Project link patterns
    project_patterns = [
        r'project:?\s*[^\n]*?(https?://[^\s]+)',
        r'(https?://github\.com/[^\s]+/[^\s]+)',
        r'(https?://gitlab\.com/[^\s]+/[^\s]+)',
        r'(https?://bitbucket\.org/[^\s]+/[^\s]+)',
    ]
    
    # Extract GitHub username
    for pattern in github_patterns:
        matches = re.findall(pattern, text)
        if matches:
            # Clean up the username (remove trailing punctuation, etc.)
            username = re.sub(r'[^a-zA-Z0-9_-]', '', matches[0])
            profiles['github'] = username
            break
    
    # Extract LeetCode username
    for pattern in leetcode_patterns:
        matches = re.findall(pattern, text)
        if matches:
            # Clean up the username
            username = re.sub(r'[^a-zA-Z0-9_-]', '', matches[0])
            profiles['leetcode'] = username
            break
    
    # Extract certification links
    certification_links = []
    for pattern in certification_patterns:
        matches = re.findall(pattern, text)
        for match in matches:
            # Clean up the URL
            url = match.rstrip(',.;:()[]{}"\'')
            certification_links.append(url)
    
    if certification_links:
        profiles['certifications'] = certification_links
    
    # Extract project links
    project_links = []
    for pattern in project_patterns:
        matches = re.findall(pattern, text)
        for match in matches:
            # Clean up the URL
            url = match.rstrip(',.;:()[]{}"\'')
            project_links.append(url)
    
    if project_links:
        profiles['projects'] = project_links
    
    return profiles

def extract_certifications(text):
    """Extract certification information from text."""
    certifications = []
    
    # Look for certification sections
    cert_section_patterns = [
        r'(?:CERTIFICATIONS?|CERTIFICATES?|QUALIFICATIONS?)[:\s]*(.*?)(?:EDUCATION|EXPERIENCE|SKILLS|PROJECTS|\Z)',
        r'(?:CERTIFICATIONS?|CERTIFICATES?)[^\n]*\n(.*?)(?:\n\s*\n|\Z)',
    ]
    
    cert_text = ""
    for pattern in cert_section_patterns:
        matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
        if matches:
            cert_text = matches[0]
            break
    
    if not cert_text:
        # Try to find individual certifications if no section was found
        individual_cert_pattern = r'(?:certified|certification|certificate)[\s:]+([^\n,]+)'
        matches = re.findall(individual_cert_pattern, text, re.IGNORECASE)
        for match in matches:
            certifications.append({
                "name": match.strip(),
                "issuer": "Unknown",
                "date": "Unknown",
                "link": None
            })
        return certifications
    
    # Split certification text into individual certifications
    cert_entries = re.split(r'\n+', cert_text)
    for entry in cert_entries:
        if len(entry.strip()) < 5:  # Skip very short lines
            continue
            
        cert = {
            "name": entry.strip(),
            "issuer": "Unknown",
            "date": "Unknown",
            "link": None
        }
        
        # Try to extract issuer
        issuer_match = re.search(r'(?:from|by|issued by|through)\s+([^,\n]+)', entry, re.IGNORECASE)
        if issuer_match:
            cert["issuer"] = issuer_match.group(1).strip()
        
        # Try to extract date
        date_match = re.search(r'(?:issued|completed|received)?\s*(?:in|on)?\s*(\d{1,2}/\d{1,2}/\d{2,4}|\w+\s+\d{4}|\d{4})', entry, re.IGNORECASE)
        if date_match:
            cert["date"] = date_match.group(1).strip()
        
        # Try to extract link
        link_match = re.search(r'(https?://[^\s]+)', entry)
        if link_match:
            cert["link"] = link_match.group(1).strip(',.;:()"\'[]{}')
        
        certifications.append(cert)
    
    return certifications

In [14]:
def create_document_parser_agent():
    """Create an improved document parser agent with enhanced profile extraction capabilities."""
    llm = ChatLiteLLM(
        model="gemini/gemini-1.5-flash",
        api_key=GEMINI_API_KEY,
        temperature=0.2
    )
    return Agent(
        role="Document Parser Specialist",
        goal="Extract comprehensive structured information from candidate resumes with a focus on profile links",
        backstory="""You are an AI specialist in parsing and extracting structured information from resume documents.
        Your expertise allows you to identify key elements like personal information, skills, experience, education,
        and public coding profiles from various document formats. You have a particular talent for finding and
        extracting URLs and profile links that may be embedded in the document, even when they are not explicitly
        labeled. You understand various resume formats and can identify GitHub, LeetCode, and certification links
        regardless of how they're presented.""",
        verbose=True,
        allow_delegation=True,
        tools=[],
        llm=llm
    )

In [15]:
def create_profile_scraper_agent():
    """Create an improved profile scraper agent."""
    llm = ChatLiteLLM(
        model="gemini/gemini-1.5-flash",
        api_key=GEMINI_API_KEY,
        temperature=0.2
    )
    return Agent(
        role="Profile Data Collector",
        goal="Gather comprehensive data from candidates' public coding profiles",
        backstory="""You specialize in collecting data from coding platforms like GitHub and LeetCode.
        Your work provides evidence of candidates' technical abilities beyond what they claim in resumes.
        You thoroughly analyze repositories, contributions, and problem-solving patterns to assess real-world
        coding abilities. You know how to extract information about languages used, project complexity,
        and development patterns from code repositories. For competitive programming profiles, you identify
        problem-solving skills and algorithmic thinking capabilities.""",
        verbose=True,
        allow_delegation=True,
        tools=[
            GitHubProfileScraper(),
            LeetCodeProfileScraper(),
            SearchTool()
        ],
        llm=llm
    )

In [16]:
def create_certification_verification_agent():
    """Create an enhanced agent for verifying certifications."""
    llm = ChatLiteLLM(
        model="gemini/gemini-1.5-flash",
        api_key=GEMINI_API_KEY,
        temperature=0.2
    )
    return Agent(
        role="Certification Verification Specialist",
        goal="Verify the authenticity and relevance of candidate certifications and map them to skills",
        backstory="""You are an expert in verifying technical certifications and credentials. 
        You know how to validate certification claims by checking certification URLs, verifying 
        credential IDs, and analyzing issuer information. You use OCR to extract text from 
        certificate images and PDFs, verify the candidate's name on certificates, and decode 
        QR codes for additional verification. You understand the significance of different 
        technical certifications across industries and can evaluate their relevance to specific 
        job roles. You can distinguish between basic completion certificates and rigorous, 
        industry-recognized credentials, providing context on the skills each certification represents.""",
        verbose=True,
        allow_delegation=True,
        tools=[
            SearchTool(),
            CertificateVerificationTool(),
            SkillMappingTool()
        ],
        llm=llm
    )

In [17]:
def create_technical_evaluator_agent():
    """Create an improved technical evaluator agent."""
    llm = ChatLiteLLM(
        model="gemini/gemini-1.5-flash",
        api_key=GEMINI_API_KEY,
        temperature=0.2
    )
    return Agent(
        role="Technical Skills Evaluator",
        goal="Assess technical proficiency based on resume claims, coding profiles, and certification evidence",
        backstory="""You are an experienced technical evaluator with deep knowledge across programming languages, frameworks,
        and software development practices. You can assess a candidate's real technical abilities by analyzing their
        coding projects, contributions, problem-solving approaches, and verified certifications. You understand the
        difference between claimed skills and demonstrated expertise, and can identify when a candidate has practical
        experience versus theoretical knowledge. You evaluate technical depth, code quality, project complexity, and 
        best practices implementation to provide a comprehensive assessment of a candidate's capabilities.""",
        verbose=True,
        allow_delegation=True,
        tools=[SearchTool()],
        llm=llm
    )

In [18]:
def create_job_alignment_agent():
    """Create an improved job alignment agent."""
    llm = ChatLiteLLM(
        model="gemini/gemini-1.5-flash",
        api_key=GEMINI_API_KEY,
        temperature=0.2
    )
    return Agent(
        role="Job Requirements Specialist",
        goal="Determine how well a candidate matches the specific technical requirements of a job",
        backstory="""With your expertise in technical recruitment, you excel at mapping candidate skills against job
        requirements. You understand both what companies need and how candidate abilities translate to on-the-job performance.
        You can identify both direct skill matches and transferable skills that might not be explicitly mentioned but are
        relevant to the role. You understand the difference between critical requirements and nice-to-have qualifications,
        weighing them appropriately in your assessment. You know how to evaluate cultural fit indicators and growth potential
        based on career progression and learning patterns shown in the candidate's profile.""",
        verbose=True,
        allow_delegation=True,
        tools=[],
        llm=llm
    )

In [19]:
def create_interview_question_generator_agent():
    """Create an improved interview question generator agent."""
    llm = ChatLiteLLM(
        model="gemini/gemini-1.5-flash",  
        api_key=GEMINI_API_KEY,
        temperature=0.3  # Slightly higher for creative questions
    )
    return Agent(
        role="Technical Interview Question Generator",
        goal="Create tailored technical interview questions based on candidate's profile and identified gaps",
        backstory="""You are an expert at crafting technical interview questions that explore the depths of a candidate's knowledge.
        Your questions help reveal actual understanding rather than memorized answers by targeting specific technologies and
        projects the candidate has worked with. You know how to design questions that assess both technical competence and
        problem-solving approach. You create a mix of questions that verify claimed expertise, explore potential skill gaps,
        and evaluate the candidate's ability to apply their knowledge in real-world scenarios. Your questions go beyond basics
        to probe depth of understanding while remaining fair and relevant to the role.""",
        verbose=True,
        allow_delegation=True,
        tools=[SearchTool()],
        llm=llm
    )

In [20]:
def create_evaluation_pipeline_agent():
    """Create a new agent for implementing the stage-by-stage evaluation pipeline."""
    llm = ChatLiteLLM(
        model="gemini/gemini-1.5-flash",
        api_key=GEMINI_API_KEY,
        temperature=0.2
    )
    return Agent(
        role="Evaluation Pipeline Manager",
        goal="Implement a comprehensive stage-by-stage evaluation process and produce a final numerical score",
        backstory="""You are a specialist in systematic candidate evaluation processes. You implement a structured
        evaluation pipeline that breaks down candidate assessment into distinct stages, each with clear criteria
        and scoring mechanisms. You understand how to weigh different aspects of a candidate's profile according
        to their relevance to the job. You produce objective, reproducible scoring that helps reduce bias in the
        hiring process while identifying candidates with the highest potential for success in the role.""",
        verbose=True,
        allow_delegation=True,
        tools=[],
        llm=llm
    )

In [21]:
def create_summary_generator_agent():
    """Create an improved summary generator agent."""
    llm = ChatLiteLLM(
        model="gemini/gemini-1.5-flash",  # Using Pro for better quality summaries
        api_key=GEMINI_API_KEY,
        temperature=0.2
    )
    return Agent(
        role="Assessment Summary Specialist",
        goal="Create engaging, concise, and visually appealing assessment reports for hiring managers",
        backstory="""You excel at synthesizing complex technical evaluations into clear, actionable, and visually engaging summaries.
        Your reports highlight candidate strengths, potential areas for growth, and relevance to specific roles
        in ways that help hiring managers make informed decisions. You have a talent for presenting information
        clearly and concisely, using visual elements like icons and rating scales to make reports easy to scan
        while still being comprehensive. You know how to create reports that are both professional and engaging,
        focusing on the most relevant insights while avoiding unnecessary details.""",
        verbose=True,
        allow_delegation=True,
        tools=[],
        llm=llm
    )


In [22]:
def create_parse_resume_task(document_parser_agent, resume_text):
    """Create an improved resume parsing task with focus on profile link extraction."""
    return Task(
        description=f"""
        Analyze the following resume text and extract structured information including:
        1. Personal information (name, contact details)
        2. Skills (programming languages, frameworks, tools)
        3. Work experience (companies, roles, responsibilities, achievements)
        4. Education (degrees, institutions, graduation dates)
        5. Projects (descriptions, technologies used)
        6. ALL public coding profile links (GitHub, LeetCode, Stack Overflow, etc.)
        7. ALL certification links, certification names, and credential IDs
        
        IMPORTANT: Search carefully for URLs and usernames that might indicate profiles on:
        - GitHub (look for github.com URLs or GitHub username mentions)
        - LeetCode (look for leetcode.com URLs or LeetCode username mentions)
        - Any certification platforms (Coursera, Udemy, AWS, Microsoft, etc.)
        
        Even if links are not explicitly labeled as profiles, extract any URLs or username mentions
        that might lead to coding profiles or certifications. Consider different text formats and
        look anywhere in the document, including headers, footers, and contact sections.
        
        Resume Text:
        {resume_text}
        
        Return the information in a structured JSON format with a dedicated "profiles" section
        and "certifications" section.
        """,
        agent=document_parser_agent,
        expected_output="A structured JSON with parsed resume information, identified coding profiles, and certifications"
    )



In [23]:
def create_scrape_profiles_task(profile_scraper_agent, profile_links):
    """Create an improved profile scraping task."""
    return Task(
        description=f"""
        Collect detailed information from the following coding profiles:
        {json.dumps(profile_links, indent=2)}
        
        For each profile:
        1. Gather comprehensive data about the candidate's activity
        2. Identify key technical metrics (languages used, projects, contributions, problem-solving ability)
        3. Look for evidence of skills claimed in their resume
        4. Analyze code quality, project complexity, and development patterns
        5. Determine the candidate's activity level and consistency of contributions
        6. Identify collaborative behaviors (such as pull requests, code reviews, etc.)
        
        For GitHub:
        - Analyze repository quality beyond just counting them
        - Examine commit patterns and contribution history
        - Check for meaningful projects vs forks or tutorial code
        - Look at code complexity and quality when possible
        
        For LeetCode:
        - Assess problem difficulty distribution (easy/medium/hard)
        - Identify algorithmic strengths and weaknesses
        - Analyze solution quality if available
        
        Return the collected information in a structured JSON format with separate sections for each platform.
        """,
        agent=profile_scraper_agent,
        expected_output="A structured JSON with detailed profile data from each platform"
    )


In [24]:
def create_verify_certifications_task(certification_verification_agent, certifications, candidate_name):
    """Create an enhanced task for verifying certifications."""
    return Task(
        description=f"""
        Verify the following certifications claimed by the candidate {candidate_name}:
        {json.dumps(certifications, indent=2)}
        
        For each certification:
        1. If a URL is provided, use the CertificateVerificationTool to:
           - Extract text from the certificate using OCR
           - Verify if the candidate's name ({candidate_name}) appears on the certificate
           - Detect and decode any QR codes on the certificate
           - Determine the issuer and legitimate skills represented by the certificate
        
        2. If no URL is provided, use the SearchTool to:
           - Research the certification to validate it exists
           - Determine what skills it typically represents
           - Identify the typical issuer and their credibility
        
        3. For each verified certification:
           - Provide a verification status (verified/unverified)
           - Include the confidence level of verification (high/medium/low)
           - List the skills demonstrated by the certification
           - Note the issuer and their credibility in the industry
           - Include issue date and expiration date if available
        
        4. After verifying all certificates, use the SkillMappingTool to map the verified certifications 
           to specific technical skills with estimated proficiency levels.
        
        Return your findings in a structured JSON format with:
        - A "certifications" section containing verification details for each certificate
        - A "skills_assessment" section mapping certificates to technical skills with proficiency ratings (1-10)
        - A "summary" section with overall assessment of the candidate's certified skills
        """,
        agent=certification_verification_agent,
        expected_output="A structured JSON with verification results for each certification and mapped skills"
    )


In [25]:
def create_evaluate_skills_task(technical_evaluator_agent, resume_data, profile_data, certification_data):
    """Create an improved skill evaluation task."""
    return Task(
        description=f"""
        Evaluate the candidate's technical skills based on:
        
        Resume Information:
        {json.dumps(resume_data, indent=2)}
        
        Profile Data:
        {json.dumps(profile_data, indent=2)}
        
        Certification Data:
        {json.dumps(certification_data, indent=2)}
        
        Provide an objective assessment of:
        1. Technical skill levels in each language/framework/tool with a 1-10 rating
        2. Evidence of practical application of claimed skills
        3. Depth of knowledge in primary areas
        4. Learning ability and adaptability based on skill acquisition timeline
        5. Areas of technical strength with specific evidence from resume/profiles/certifications
        6. Potential growth areas or skill gaps
        7. Specialization vs. generalist assessment
        
        For each skill:
        - Distinguish between claimed skills and demonstrated skills
        - Consider both experience (years) and evidence of mastery
        - Weigh profile evidence more heavily than resume claims
        - Consider certification difficulty and relevance
        
        Return your evaluation in a structured JSON format with skills grouped by category
        (languages, frameworks, tools, etc.) and each assigned a numerical rating.
        """,
        agent=technical_evaluator_agent,
        expected_output="A structured JSON with detailed technical skill evaluation"
    )



In [26]:
def create_job_match_task(job_alignment_agent, skill_evaluation, job_requirements):
    """Create an improved job match task."""
    return Task(
        description=f"""
        Determine how well the candidate matches the following job requirements:
        {job_requirements}
        
        Using their skill evaluation:
        {json.dumps(skill_evaluation, indent=2)}
        
        Assess:
        1. Essential requirements match (must-have skills)
        2. Preferred requirements match (nice-to-have skills)
        3. Experience level alignment
        4. Cultural fit indicators
        5. Growth potential for the role
        
        For each requirement:
        - Identify if it's satisfied directly or through a transferable skill
        - Consider not just the presence of a skill but its rated level
        - Note where the candidate exceeds requirements
        - Note where the candidate falls short but may be able to learn
        - Note where critical gaps exist
        
        Provide a match percentage (0-100%) and detailed explanation for each major requirement.
        Also calculate an overall match percentage that weights essential requirements more heavily.
        Return your assessment in a structured JSON format.
        """,
        agent=job_alignment_agent,
        expected_output="A structured JSON with detailed job requirement matches"
    )



In [27]:
def create_evaluation_pipeline_task(evaluation_pipeline_agent, resume_data, profile_data, certification_data, skill_evaluation, job_match):
    """Create a new task for the stage-by-stage evaluation pipeline."""
    return Task(
        description=f"""
        Implement a comprehensive evaluation pipeline with the following stages, using:
        
        Resume Information:
        {json.dumps(resume_data, indent=2)}
        
        Profile Data:
        {json.dumps(profile_data, indent=2)}
        
        Certification Data:
        {json.dumps(certification_data, indent=2)}
        
        Skill Evaluation:
        {json.dumps(skill_evaluation, indent=2)}
        
        Job Match Assessment:
        {json.dumps(job_match, indent=2)}
        
        **Stage 1: Basic Eligibility Check** (20 points)
        - Extract and verify: education, years of experience, certifications, location (if required)
        - Calculate points based on minimum requirements being met
        
        **Stage 2: Skill Match Evaluation** (40 points)
        - Use the skill evaluation and job match data
        - Classify: Matched Skills, Missing Skills, Bonus Skills
        - Weight essential skills higher than preferred skills
        
        **Stage 3: Domain-Specific Experience Check** (15 points)
        - Identify domain-related keywords from the job requirements
        - Score based on evidence of domain experience in resume and profiles
        
        **Stage 4: Role Fit and Achievements Alignment** (25 points)
        - Assess alignment of resume achievements with job responsibilities
        - Prioritize impact-driven achievements and leadership experience if relevant
        
        For each stage:
        - Provide a numeric score
        - Include a justification for the score
        - Highlight key factors that influenced the score
        
        Finally:
        - Calculate a total score out of 100
        - Provide a final recommendation: Proceed to interview / Hold / Reject
          (Proceed: 70+, Hold: 50-69, Reject: <50)
        
        Return a structured JSON with results for each stage, total score, and recommendation.
        """,
        agent=evaluation_pipeline_agent,
        expected_output="A structured JSON containing scores and evaluations for each stage of the pipeline"
    )



In [28]:
def create_generate_questions_task(interview_question_generator, resume_data, profile_data, skill_evaluation, job_match):
    """Create an improved interview question generation task."""
    return Task(
        description=f"""
        Generate strategic technical interview questions based on:
        
        Resume Information:
        {json.dumps(resume_data, indent=2)}
        
        Profile Data:
        {json.dumps(profile_data, indent=2)}
        
        Skill Evaluation:
        {json.dumps(skill_evaluation, indent=2)}
        
        Job Match Assessment:
        {json.dumps(job_match, indent=2)}
        
        Create 5-7 tailored questions that:
        1. Verify depth of knowledge in claimed expertise areas
        2. Explore specific projects mentioned in resume or profiles
        3. Address identified skill gaps relevant to the job requirements
        4. Include at least one system design question related to their experience
        5. Include at least one problem-solving question that relates to their domain
        6. Test adaptability by asking about unfamiliar but related technologies
        
        For each question:
        - Make it specific to this candidate (reference specific projects or experience)
        - Clearly state what you're assessing with this question
        - Provide guidance on what a good answer would include
        - Structure questions that reveal thinking process, not just knowledge
        - Include difficulty level (Basic, Intermediate, Advanced)
        
        Return the questions in a structured JSON format.
        """,
        agent=interview_question_generator,
        expected_output="A structured JSON with tailored technical interview questions"
    )



In [29]:
def create_summary_task(summary_generator, resume_data, profile_data, skill_evaluation, job_match, interview_questions, evaluation_results):
    """Create an improved summary generation task with focus on readability and visual appeal."""
    return Task(
        description=f"""
        Create a compelling, visually engaging, and concise assessment summary based on:
        
        Resume Information:
        {json.dumps(resume_data, indent=2)}
        
        Profile Data:
        {json.dumps(profile_data, indent=2)}
        
        Skill Evaluation:
        {json.dumps(skill_evaluation, indent=2)}
        
        Job Match Assessment:
        {json.dumps(job_match, indent=2)}
        
        Suggested Interview Questions:
        {json.dumps(interview_questions, indent=2)}
        
        Evaluation Results:
        {json.dumps(evaluation_results, indent=2)}
        
        Create a summary that:
        1. Starts with a brief candidate overview (name, current role, years of experience)
        2. Includes a visually appealing score card with the overall evaluation score
        3. Highlights 3-5 key strengths with evidence from their resume and profiles
        4. Notes 2-3 potential areas for growth relevant to the job
        5. Summarizes overall job match with a clear recommendation
        6. Uses visually engaging elements like:
           - Star ratings (★★★★☆) for key skills
           - Emojis as bullet points for visual scanning
           - Rating scales for key metrics
           - Clear headings and subheadings
        
        The summary should be scannable in 30 seconds while providing valuable insights.
        Format in visually appealing markdown, optimized for a hiring manager who has limited time.
        Keep the entire summary under 2 pages when printed.
        """,
        agent=summary_generator,
        expected_output="A concise, visually engaging markdown assessment summary"
    )



In [30]:
def run_skill_assessment(resume_file, job_requirements):
    print("Starting enhanced skill assessment process...")
    
    # Step 1: Extract text from resume document
    print("Extracting text from resume...")
    resume_text = extract_text_from_document(resume_file)
    
    # Step 2: Create agents
    print("Creating specialized agents...")
    document_parser_agent = create_document_parser_agent()
    profile_scraper_agent = create_profile_scraper_agent()
    certification_verification_agent = create_certification_verification_agent()
    technical_evaluator_agent = create_technical_evaluator_agent()
    job_alignment_agent = create_job_alignment_agent()
    evaluation_pipeline_agent = create_evaluation_pipeline_agent()
    interview_question_generator = create_interview_question_generator_agent()
    summary_generator_agent = create_summary_generator_agent()
    
    # Step 3: Define tasks and create parsing crew
    print("Creating assessment tasks...")
    parse_resume_task = create_parse_resume_task(document_parser_agent, resume_text)
    
    # Step 4: Create crew and execute
    print("Executing document parsing...")
    parsing_crew = Crew(
        agents=[document_parser_agent],
        tasks=[parse_resume_task],
        verbose=True,
        process=Process.sequential
    )
    
    resume_result = parsing_crew.kickoff()
    print("\nResume parsing complete!")
    
    try:
        # Access the actual string value from the CrewOutput object
        resume_result_str = str(resume_result)
        resume_data = json.loads(resume_result_str)
    except json.JSONDecodeError:
        # Try to extract JSON from text if the result isn't directly parseable
        try:
            json_match = re.search(r'```json\n(.*?)\n```', resume_result_str, re.DOTALL)
            if json_match:
                resume_data = json.loads(json_match.group(1))
            else:
                print("Warning: Could not parse resume result as JSON. Using raw text.")
                resume_data = {"raw_result": resume_result_str}
        except Exception as e:
            print(f"Error parsing resume JSON: {str(e)}")
            resume_data = {"raw_result": resume_result_str}
    
    # Extract candidate name for certificate verification
    candidate_name = extract_name_from_resume(resume_data)
    print(f"\nCandidate name identified: {candidate_name}")
    
    # Step 5: Extract profile links and scrape profiles
    print("\nExtracting profile links...")
    if isinstance(resume_data, dict) and "raw_result" not in resume_data:
        # Use the structured data if available
        profile_links = resume_data.get("profiles", {})
        if not profile_links:
            # Try to find links in the structured data
            profile_links = extract_profile_links(str(resume_data))
    else:
        # Use the raw text for extraction
        profile_links = extract_profile_links(resume_text)
    
    print(f"Found profile links: {profile_links}")
    
    if profile_links:
        print("\nScraping coding profiles...")
        scrape_profiles_task = create_scrape_profiles_task(profile_scraper_agent, profile_links)
        
        profile_crew = Crew(
            agents=[profile_scraper_agent],
            tasks=[scrape_profiles_task],
            verbose=True,
            process=Process.sequential
        )
        
        profile_result = profile_crew.kickoff()
        print("\nProfile scraping complete!")
        
        try:
            # Convert CrewOutput to string before parsing as JSON
            profile_result_str = str(profile_result)
            profile_data = json.loads(profile_result_str)
        except json.JSONDecodeError:
            # Try to extract JSON from text
            try:
                json_match = re.search(r'```json\n(.*?)\n```', profile_result_str, re.DOTALL)
                if json_match:
                    profile_data = json.loads(json_match.group(1))
                else:
                    print("Warning: Could not parse profile result as JSON. Using raw text.")
                    profile_data = {"raw_result": profile_result_str}
            except Exception as e:
                print(f"Error parsing profile JSON: {str(e)}")
                profile_data = {"raw_result": profile_result_str}
    else:
        print("No coding profiles found. Proceeding with resume data only.")
        profile_data = {}
    
    # Step 6: Process and verify certifications
    print("\nExtracting certification information...")
    certifications = []
    
    # First check for structured certification data in resume_data
    if isinstance(resume_data, dict) and "certifications" in resume_data:
        cert_data = resume_data["certifications"]
        if isinstance(cert_data, list):
            certifications = cert_data
        elif isinstance(cert_data, dict):
            # Convert dict to list if needed
            certifications = [cert_data] if cert_data else []
    
    # If no structured data, try to extract from raw text
    if not certifications:
        certifications = extract_certifications(resume_text)
    
    print(f"Found {len(certifications)} certifications")
    
    if certifications:
        print("\nVerifying certifications...")
        verify_certifications_task = create_verify_certifications_task(
            certification_verification_agent, 
            certifications,
            candidate_name
        )
        
        certification_crew = Crew(
            agents=[certification_verification_agent],
            tasks=[verify_certifications_task],
            verbose=True,
            process=Process.sequential
        )
        
        certification_result = certification_crew.kickoff()
        print("\nCertification verification complete!")
        
        try:
            certification_result_str = str(certification_result)
            certification_data = json.loads(certification_result_str)
        except json.JSONDecodeError:
            try:
                json_match = re.search(r'```json\n(.*?)\n```', certification_result_str, re.DOTALL)
                if json_match:
                    certification_data = json.loads(json_match.group(1))
                else:
                    print("Warning: Could not parse certification result as JSON. Using raw text.")
                    certification_data = {"raw_result": certification_result_str}
            except Exception as e:
                print(f"Error parsing certification JSON: {str(e)}")
                certification_data = {"raw_result": certification_result_str}
    else:
        print("No certifications found. Proceeding without certification verification.")
        certification_data = {}
    
    # Step 7: Evaluate technical skills
    print("\nEvaluating technical skills...")
    evaluate_skills_task = create_evaluate_skills_task(technical_evaluator_agent, resume_data, profile_data, certification_data)
    
    skills_crew = Crew(
        agents=[technical_evaluator_agent],
        tasks=[evaluate_skills_task],
        verbose=True,
        process=Process.sequential
    )
    
    skills_result = skills_crew.kickoff()
    print("\nSkill evaluation complete!")
    
    try:
        # Convert CrewOutput to string before parsing as JSON
        skills_result_str = str(skills_result)
        skill_evaluation = json.loads(skills_result_str)
    except json.JSONDecodeError:
        # Try to extract JSON from text
        try:
            json_match = re.search(r'```json\n(.*?)\n```', skills_result_str, re.DOTALL)
            if json_match:
                skill_evaluation = json.loads(json_match.group(1))
            else:
                print("Warning: Could not parse skill evaluation as JSON. Using raw text.")
                skill_evaluation = {"raw_result": skills_result_str}
        except Exception as e:
            print(f"Error parsing skill evaluation JSON: {str(e)}")
            skill_evaluation = {"raw_result": skills_result_str}
    
    # Step 8: Assess job match
    print("\nAssessing job match...")
    job_match_task = create_job_match_task(job_alignment_agent, skill_evaluation, job_requirements)
    
    job_match_crew = Crew(
        agents=[job_alignment_agent],
        tasks=[job_match_task],
        verbose=True,
        process=Process.sequential
    )
    
    job_match_result = job_match_crew.kickoff()
    print("\nJob match assessment complete!")
    
    try:
        # Convert CrewOutput to string before parsing as JSON
        job_match_result_str = str(job_match_result)
        job_match = json.loads(job_match_result_str)
    except json.JSONDecodeError:
        # Try to extract JSON from text
        try:
            json_match = re.search(r'```json\n(.*?)\n```', job_match_result_str, re.DOTALL)
            if json_match:
                job_match = json.loads(json_match.group(1))
            else:
                print("Warning: Could not parse job match as JSON. Using raw text.")
                job_match = {"raw_result": job_match_result_str}
        except Exception as e:
            print(f"Error parsing job match JSON: {str(e)}")
            job_match = {"raw_result": job_match_result_str}
    
    # Step 9: Run the evaluation pipeline
    print("\nRunning evaluation pipeline...")
    evaluation_pipeline_task = create_evaluation_pipeline_task(
        evaluation_pipeline_agent,
        resume_data,
        profile_data,
        certification_data,
        skill_evaluation,
        job_match
    )
    
    evaluation_crew = Crew(
        agents=[evaluation_pipeline_agent],
        tasks=[evaluation_pipeline_task],
        verbose=True,
        process=Process.sequential
    )
    
    evaluation_result = evaluation_crew.kickoff()
    print("\nEvaluation pipeline complete!")
    
    try:
        evaluation_result_str = str(evaluation_result)
        evaluation_results = json.loads(evaluation_result_str)
    except json.JSONDecodeError:
        try:
            json_match = re.search(r'```json\n(.*?)\n```', evaluation_result_str, re.DOTALL)
            if json_match:
                evaluation_results = json.loads(json_match.group(1))
            else:
                print("Warning: Could not parse evaluation results as JSON. Using raw text.")
                evaluation_results = {"raw_result": evaluation_result_str}
        except Exception as e:
            print(f"Error parsing evaluation results JSON: {str(e)}")
            evaluation_results = {"raw_result": evaluation_result_str}
    
    # Step 10: Generate interview questions
    print("\nGenerating interview questions...")
    questions_task = create_generate_questions_task(
        interview_question_generator, 
        resume_data, 
        profile_data,
        skill_evaluation,
        job_match
    )
    
    questions_crew = Crew(
        agents=[interview_question_generator],
        tasks=[questions_task],
        verbose=True,
        process=Process.sequential
    )
    
    questions_result = questions_crew.kickoff()
    print("\nInterview questions generated!")
    
    try:
        # Convert CrewOutput to string before parsing as JSON
        questions_result_str = str(questions_result)
        interview_questions = json.loads(questions_result_str)
    except json.JSONDecodeError:
        # Try to extract JSON from text
        try:
            json_match = re.search(r'```json\n(.*?)\n```', questions_result_str, re.DOTALL)
            if json_match:
                interview_questions = json.loads(json_match.group(1))
            else:
                print("Warning: Could not parse interview questions as JSON. Using raw text.")
                interview_questions = {"raw_result": questions_result_str}
        except Exception as e:
            print(f"Error parsing interview questions JSON: {str(e)}")
            interview_questions = {"raw_result": questions_result_str}
    
    # Step 11: Generate final summary
    print("\nGenerating final assessment summary...")
    summary_task = create_summary_task(
        summary_generator_agent,
        resume_data,
        profile_data,
        skill_evaluation,
        job_match,
        interview_questions,
        evaluation_results
    )
    
    summary_crew = Crew(
        agents=[summary_generator_agent],
        tasks=[summary_task],
        verbose=True,
        process=Process.sequential
    )
    
    summary_result = summary_crew.kickoff()
    print("\nFinal assessment summary generated!")
    
    # Step 12: Return all results
    results = {
        "resume_data": resume_data,
        "profile_data": profile_data,
        "certification_data": certification_data,
        "skill_evaluation": skill_evaluation,
        "job_match": job_match,
        "evaluation_results": evaluation_results,
        "interview_questions": interview_questions,
        "summary": summary_result
    }
    
    return results

In [31]:

# Create sample job requirements
sample_job_requirements = """
Job Title: Senior Full Stack Developer

Essential Requirements:
- 5+ years of experience in software development
- Strong proficiency in Python and JavaScript
- Experience with React.js and Django
- Knowledge of RESTful API design and implementation
- Experience with cloud platforms (AWS or Azure)
- Understanding of CI/CD pipelines and DevOps practices

Preferred Skills:
- Experience with TypeScript and Node.js
- Knowledge of container technologies (Docker, Kubernetes)
- Experience with database design and optimization
- Familiarity with microservices architecture
- Experience mentoring junior developers

Responsibilities:
- Design, develop, and maintain web applications
- Collaborate with cross-functional teams to define and implement new features
- Write clean, maintainable, and efficient code
- Participate in code reviews and provide constructive feedback
- Troubleshoot production issues and optimize application performance
- Stay up-to-date with emerging trends and technologies
"""

In [32]:
with open("Nandini_kuppala_latest.pdf", "rb") as file:
    results = run_skill_assessment(file, sample_job_requirements)

Starting enhanced skill assessment process...
Extracting text from resume...
Creating specialized agents...
Creating assessment tasks...
Executing document parsing...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a9861f53-02a1-4ddc-91bd-b21c1d7b3bf2                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 07919dd7-7602-490e-8191-3195229525d8
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 07919dd7-7602-490e-8191-3195229525d8
       Status: Executing Task...
    └── 🤖 Agent: Document Parser Specialist
            Status: In Progress

# Agent: Document Parser Specialist
## Task: 
        Analyze the following resume text and extract structured information including:
        1. Personal information (name, contact details)
        2. Skills (programming languages, frameworks, tools)
        3. Work experience (companies, roles, responsibilities, achievements)
        4. Education (degrees, institutions, graduation dates)
        5. Projects (descriptions, technologies used)
        6. ALL public coding profile links (GitHub, LeetCode, Stack Overflow, etc.)
        7. ALL certification links, certification names, and credential IDs

        IMPORTANT: Search carefully for URLs and usernames that might indicate profiles on:
        - GitHub (look for github.com URLs or GitHub username mentions)
        - LeetCode (look for leetcode.com URLs or LeetCode username mentions)
        - Any certification platforms (Coursera, Udemy, AWS, Microsoft, etc.)

        Even if links are not explicitly labeled as profiles, extract an

...

🚀 Crew: crew
└── 📋 Task: 7d8ae262-63f3-48d1-a6ad-1c1229f1ca05
       Status: Executing Task...
    └── 🤖 Agent: Assessment Summary Specialist
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 7d8ae262-63f3-48d1-a6ad-1c1229f1ca05
       Assigned to: Assessment Summary Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Assessment Summary Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7d8ae262-63f3-48d1-a6ad-1c1229f1ca05                                                                     │
│  Agent: Assessment Summary Specialist                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: cdcf8647-1fd7-44c1-ab40-00e15b51a4c5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Final assessment summary generated!


In [33]:
print(results)

{'resume_data': {'personal_information': {'name': 'NANDINI KUPPALA', 'contact': {'phone': '+91 7569056212', 'email': 'knandini7816@gmail.com', 'linkedin': 'https://www.linkedin.com/in/nandini-kuppala/', 'github': 'https://github.com/nandini-kuppala'}}, 'skills': {'languages': ['Python', 'Dart', 'JavaScript', 'C'], 'frameworks': ['Flutter', 'Flask', 'Streamlit'], 'tools': ['Git', 'Docker', 'Selenium', 'Firebase', 'MySQL', 'MongoDB', 'Linux'], 'ai_and_development': ['Generative AI', 'ML', 'DL', 'NLP', 'Computer Vision', 'Data Structures', 'Agile', 'Testing']}, 'work_experience': [{'title': 'Infosys Springboard Internship 5.O (AI Project Intern)', 'company': 'Infosys', 'dates': 'December 2024 - Present', 'responsibilities': ['Led a team of five interns and built an AI system using NLP to automate ticket classification and responses.', 'Integrated with helpdesk platforms to enhance efficiency and reduce resolution time.']}, {'title': 'Research Experience', 'company': None, 'dates': 'Octobe


# Nandini Kuppala - Assessment Summary

**Candidate Overview:** Nandini Kuppala is an aspiring AI Engineer currently completing a Bachelor of Engineering in Artificial Intelligence.  While her resume doesn't explicitly state years of experience, her extensive project portfolio suggests a strong foundation equivalent to 5+ years of experience.

## Overall Evaluation Score: 75/100 👍

```
| Category                     | Score | Rating |
|------------------------------|-------|---------|
| Basic Eligibility Check      | 15/20 | ★★★☆☆  |
| Skill Match Evaluation       | 28/40 | ★★★★☆ |
| Domain-Specific Experience   | 10/15 | ★★☆☆☆  |
| Role Fit & Achievements     | 22/25 | ★★★★☆ |
```

## Key Strengths 🌟

* **Strong Programming Skills:**  Nandini demonstrates proficiency in Python (★★★★☆), Dart (★★★☆☆), and JavaScript (★★★☆☆), evidenced by her projects (NurtureSync, DeepFake detection), certifications (HackerRank, Coursera), and LeetCode performance (272 problems solved).
* **Proven AI/ML Expertise:**  Her projects showcase practical application of Machine Learning (★★★★☆), Deep Learning (★★★☆☆), and Natural Language Processing (★★★★☆).  Her research experience further solidifies her expertise in these areas.
* **Successful Project Deployments:**  Nandini successfully deployed the NurtureSync healthcare app (web and mobile), demonstrating her ability to translate ideas into functional, user-friendly applications.
* **Effective Testing Abilities:**  Her internship at Baavlibuch highlights her skills in automated testing (Selenium), resulting in a 40% reduction in manual testing effort and a 25% acceleration in test case execution.
* **Publication Record:**  Nandini has published research in prestigious venues (ASPRS, IEEE), showcasing her ability to contribute to the field and communicate complex technical information effectively.


## Areas for Growth 🌱

* **Expand Cloud Platform Experience:**  Gaining experience with AWS or Azure is crucial for many roles and will significantly enhance her skillset.
* **Deepen Frontend Frameworks:**  While proficient in Flask and Streamlit, exploring React.js or similar frameworks would broaden her full-stack capabilities.
* **Master CI/CD Pipelines:**  Familiarizing herself with CI/CD practices and tools like Jenkins will be highly beneficial for future roles.


## Overall Job Match & Recommendation

Nandini's overall match for the Senior Full Stack Developer role is currently 45%, primarily due to gaps in essential technologies (React.js, Django, cloud platforms, CI/CD). However, her strong foundation, impressive project portfolio, and demonstrable learning agility make her a promising candidate with high growth potential (90% match).  We recommend proceeding to the interview stage, focusing on assessing her ability to quickly acquire the missing skills and her understanding of the gaps.  The interview should delve into the depth of her existing projects to better gauge her experience level.  Her cultural fit indicators are strong (80% match).[00m

## Document Parser Specialist Output
```json
{
  "personal_information": {
    "name": "NANDINI KUPPALA",
    "contact": {
      "phone": "+91 7569056212",
      "email": "knandini7816@gmail.com",
      "linkedin": "https://www.linkedin.com/in/nandini-kuppala/"
    }
  },
  "skills": {
    "languages": ["Python", "Dart", "JavaScript", "C"],
    "frameworks": ["Flutter", "Flask", "Streamlit"],
    "tools": ["Git", "Docker", "Selenium", "Firebase", "MySQL", "MongoDB", "Linux"],
    "ai_and_development": ["Generative AI", "ML", "DL", "NLP", "Computer Vision", "Data Structures", "Agile", "Testing"]
  },
  "experience": [
    {
      "title": "Infosys Springboard Internship 5.O (AI Project Intern)",
      "company": "Infosys",
      "dates": "December 2024 - Present",
      "responsibilities": [
        "Led a team of five interns and built an AI system using NLP to automate ticket classification and responses.",
        "Integrated with helpdesk platforms to enhance efficiency and reduce resolution time."
      ]
    },
    {
      "title": "Research Experience",
      "company": null,
      "dates": "October 2023 – Present",
      "mentors": ["Dr. Ramesh Sivanpillai (University of Wyoming)", "Dr. Sajith Variyar (Amrita School of AI)"],
      "responsibilities": [
        "Worked on real-time machine learning projects using LANDSAT data to monitor rangeland vegetation.",
        "Contributed to a 30% increase in profitability for Wyoming farmers."
      ]
    },
    {
      "title": "Campus Ambassador",
      "company": "E-Cell IIT Bombay",
      "dates": "August 2024 – December 2024",
      "responsibilities": [
        "Organized events and competitions reaching over 200 participants, promoting entrepreneurship in college."
      ]
    },
    {
      "title": "Security and Software Testing Intern",
      "company": "Baavlibuch",
      "dates": "February 2024 – April 2024",
      "responsibilities": [
        "Automated testing procedures, reducing manual testing effort by 40% and accelerating test case execution by 25%.",
        "Debugged and fixed critical application bugs, enhancing stability and performance by 30%.",
        "Implemented and managed testing environments in Linux, significantly improving security and data protection."
      ]
    }
  ],
  "education": [
    {
      "degree": "Bachelor of Engineering in Artificial Intelligence",
      "institution": "Amrita Vishwa Vidyapeetham",
      "location": "Coimbatore, Tamil Nadu",
      "cgpa": "8.02",
      "dates": "2022 – 2026"
    }
  ],
  "projects": [
    {
      "name": "NurtureSync: A Health Care Companion",
      "description": "Developed a mobile and web app platform to manage chronic diseases like thyroid and diabetes. Features include personalized feedback, medical report analysis, health tracking, and a healthcare community.",
      "technologies": ["Web – Streamlit, MongoDB, Firebase, Flask; Mobile – Flutter, Firebase, MongoDB"],
      "link": "https://nurturesynchealth.streamlit.app/"
    },
    {
      "name": "DeepFake Video Detection",
      "description": "Improved the model’s accuracy by 15% in detecting DeepFake videos using a pipeline of MTCNN for face detection, ResNet for feature extraction, and LSTM for temporal analysis. Developed a Chrome extension for DeepFake image detection and working towards real-time video detection",
      "technologies": ["MTCNN", "ResNet", "LSTM"],
      "link": "https://github.com/nandini-queen-of-my-world/Deepfake-detection"
    }
  ],
  "achievements": [
    "Finalist at the Health Hackathon by Johns Hopkins University at VIT Bhopal, selected from 232 teams and over 1,300 participants for the project NurtureSync."
  ],
  "publications": [
    {
      "title": "Mapping Vegetation Dynamics in Wyoming: A Multi-Temporal Analysis Using Landsat NDVI and Clustering",
      "venue": "ASPRS International Technical Symposium 2024",
      "link": "https://isprs-archives.copernicus.org/articles/XLVIII-M-5-2024/87/2025/"
    },
    {
      "title": "Preterm Birth Prediction Using GraphSAGE and Electrohysterogram Data",
      "venue": "IEEE Eleventh International Conference on Biosignals, Images, and Instrumentation (ICBSII)",
      "link": "https://isprs-archives.copernicus.org/articles/XLVIII-M-5-2024/87/2025/"
    }
  ],
  "profiles": {
    "github": "https://github.com/nandini-kuppala",
    "leetcode": "https://leetcode.com/u/Nandini_Queen/",
    "geeksforgeeks": "https://www.geeksforgeeks.org/user/knandin8fsf/",
    "hackerrank": "https://www.hackerrank.com/profile/knandini7816"
  },
  "certifications": [
    {
      "name": "Artificial Intelligence Course Series (118 hr length)",
      "issuer": "Infosys Springboard",
      "link": "https://infyspringboard.onwingspan.com/web/en/app/profile/competency/certificate"
    },
    {
      "name": "Agile Scrum in Practice",
      "issuer": "Infosys Springboard",
      "link": "https://infyspringboard.onwingspan.com/web/en/app/profile/competency/certificate"
    },
    {
      "name": "Data Structures and Performance",
      "issuer": "Coursera",
      "link": "https://coursera.org/share/1d4edbf7bd94d9c9409693f3ed34623e"
    },
    {
      "name": "Python Basic, Problem Solving Basic",
      "issuer": "Hacker Rank",
      "link": "https://www.hackerrank.com/certificates/29cd025a29fc"
    }
  ]
}
```

## Profile Data Collector
```json
{
  "github": {
    "username": "nandini-kuppala",
    "name": "Nandini ",
    "bio": "Aspiring AI Engineer",
    "public_repos": 30,
    "followers": 10,
    "following": 11,
    "top_languages": [
      "Jupyter Notebook",
      "Python",
      "HTML",
      "JavaScript",
      "C++"
    ],
    "starred_repos_count": 9,
    "forks_count": 0,
    "profile_created_at": "2022-12-12T17:12:54Z"
  },
  "leetcode": {
    "username": "Nandini_Queen",
    "total_problems_solved": 272,
    "problems_by_difficulty": {
      "All": 136,
      "Easy": 58,
      "Medium": 68,
      "Hard": 10
    },
    "ranking": 883273,
    "reputation": 0,
    "badges": [
      "Study Plan V2 Award"
    ]
  },
  "geeksforgeeks": "Data not available",
  "hackerrank": "Data not available"
}```

## Certification Verification Specialist
```json
{
  "certifications": [
    {
      "name": "Artificial Intelligence Course Series (118 hr length)",
      "provider": "Infosys Springboard",
      "link": "https://infyspringboard.onwingspan.com/web/en/app/profile/competency/certificate",
      "verificationStatus": "Partial",
      "notes": "Link provided leads to a generic profile page.  Cannot verify certificate details or candidate ownership without access to a specific candidate profile.  The course suggests skills in AI concepts and techniques."
    },
    {
      "name": "Agile Scrum in Practice",
      "provider": "Infosys Springboard",
      "link": "https://infyspringboard.onwingspan.com/web/en/app/profile/competency/certificate",
      "verificationStatus": "Partial",
      "notes": "Link provided leads to a generic profile page. Cannot verify certificate details or candidate ownership without access to a specific candidate profile.  The course suggests skills in Agile methodologies and Scrum framework."
    },
    {
      "name": "Data Structures and Performance",
      "provider": "Coursera",
      "link": "https://coursera.org/share/1d4edbf7bd94d9c9409693f3ed34623e",
      "verificationStatus": "Partial",
      "notes": "Unable to fully verify due to tool errors.  The course title suggests skills in data structures and algorithm analysis."
    },
    {
      "name": "Python Basic, Problem Solving Basic",
      "provider": "HackerRank",
      "link": "https://www.hackerrank.com/certificates/29cd025a29fc",
      "verificationStatus": "Partial",
      "notes": "Unable to fully verify due to tool errors.  The certificate suggests basic skills in Python programming and problem-solving."
    }
  ]
}
```

## Technical Skills Evaluator
```json
{
  "languages": {
    "Python": {
      "rating": 8,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Used extensively in multiple projects (NurtureSync, DeepFake detection), HackerRank certification, LeetCode problem-solving.",
      "notes": "Demonstrates proficiency through projects and certifications. LeetCode performance further supports this."
    },
    "Dart": {
      "rating": 7,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Used in Flutter mobile app development for NurtureSync.",
      "notes": "Demonstrated use in a project, but further evidence of mastery would strengthen the rating."
    },
    "JavaScript": {
      "rating": 6,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Used in web development for NurtureSync, GitHub profile shows usage.",
      "notes": "Evidence suggests familiarity, but deeper projects would improve the rating."
    },
    "C": {
      "rating": 5,
      "claimed": true,
      "demonstrated": false,
      "evidence": "Mentioned in resume, but no project evidence.",
      "notes": "Requires further evidence of practical application."
    }
  },
  "frameworks": {
    "Flutter": {
      "rating": 7,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Used for mobile app development in NurtureSync.",
      "notes": "Successful project deployment shows competency."
    },
    "Flask": {
      "rating": 6,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Used in NurtureSync web app backend.",
      "notes": "Demonstrates basic understanding, more complex applications would improve the rating."
    },
    "Streamlit": {
      "rating": 6,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Used in NurtureSync web app frontend.",
      "notes": "Demonstrates basic understanding, more complex applications would improve the rating."
    }
  },
  "tools": {
    "Git": {
      "rating": 8,
      "claimed": true,
      "demonstrated": true,
      "evidence": "GitHub profile indicates active use.",
      "notes": "Active GitHub profile suggests good understanding and usage."
    },
    "Docker": {
      "rating": 5,
      "claimed": true,
      "demonstrated": false,
      "evidence": "Mentioned in resume, but no project evidence.",
      "notes": "Requires further evidence of practical application."
    },
    "Selenium": {
      "rating": 7,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Used in Baavlibuch internship for automation testing.",
      "notes": "Practical application demonstrated in internship."
    },
    "Firebase": {
      "rating": 7,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Used in NurtureSync for backend services.",
      "notes": "Successful integration in a project shows competency."
    },
    "MySQL": {
      "rating": 5,
      "claimed": true,
      "demonstrated": false,
      "evidence": "Mentioned in resume, but no project evidence.",
      "notes": "Requires further evidence of practical application."
    },
    "MongoDB": {
      "rating": 7,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Used in NurtureSync for database management.",
      "notes": "Successful integration in a project shows competency."
    },
    "Linux": {
      "rating": 6,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Used in Baavlibuch internship for testing environments.",
      "notes": "Practical application demonstrated in internship, but more complex usage would improve the rating."
    }
  },
  "AI and Development": {
    "Generative AI": {
      "rating": 5,
      "claimed": true,
      "demonstrated": false,
      "evidence": "Mentioned in resume, but no specific project evidence.",
      "notes": "Requires further evidence of practical application."
    },
    "ML": {
      "rating": 8,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Used in research projects and DeepFake detection.",
      "notes": "Strong evidence of practical application in research and projects."
    },
    "DL": {
      "rating": 7,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Used in DeepFake detection project.",
      "notes": "Demonstrated use in a project, but further evidence of mastery would strengthen the rating."
    },
    "NLP": {
      "rating": 8,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Used in Infosys internship for ticket classification.",
      "notes": "Successful application in a significant project."
    },
    "Computer Vision": {
      "rating": 7,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Used in DeepFake detection project.",
      "notes": "Demonstrated use in a project, but further evidence of mastery would strengthen the rating."
    },
    "Data Structures": {
      "rating": 7,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Coursera certification, LeetCode problem-solving.",
      "notes": "Certification and LeetCode performance suggest good understanding."
    },
    "Agile": {
      "rating": 6,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Infosys Springboard certification, experience in team projects.",
      "notes": "Certification and project experience suggest familiarity, but more complex projects would improve the rating."
    },
    "Testing": {
      "rating": 7,
      "claimed": true,
      "demonstrated": true,
      "evidence": "Baavlibuch internship experience, significant results reported.",
      "notes": "Strong evidence of practical application and results."
    }
  },
  "overallAssessment": {
    "practicalApplication": "Strong evidence of practical application across multiple projects and internships.  The candidate demonstrates the ability to apply learned skills to solve real-world problems.",
    "depthOfKnowledge": "Demonstrates good depth of knowledge in several areas, particularly in ML, NLP, and Python.  However, some areas like Generative AI require further exploration.",
    "learningAbility": "The candidate's skill acquisition timeline and diverse project portfolio suggest a strong learning ability and adaptability.  The candidate actively seeks new challenges and technologies.",
    "strengths": "Strong programming skills (Python, Dart), experience in ML/DL/NLP, successful project deployments (NurtureSync), and proven testing abilities.",
    "growthAreas": "Expand experience with Generative AI, explore more complex projects using Docker and MySQL, and deepen expertise in specific AI subfields.",
    "specializationVsGeneralist": "The candidate is currently a generalist with a strong focus on AI and its applications.  Further specialization in a specific AI subfield (e.g., NLP or Computer Vision) could enhance career prospects."
  }
}
```

## Job Requirements Specialist
```json
{
  "assessment": {
    "essential_requirements_match": {
      "Python": {
        "match_percentage": 90,
        "explanation": "Candidate demonstrates excellent proficiency in Python (level 8), exceeding the requirement.  However, the job description requires experience with Django, which is missing."
      },
      "JavaScript": {
        "match_percentage": 40,
        "explanation": "Candidate's JavaScript skills are rated at level 5, which is below the required 'strong proficiency'. Further evidence is needed to confirm proficiency."
      },
      "React.js": {
        "match_percentage": 0,
        "explanation": "No experience with React.js is mentioned in the candidate's evaluation."
      },
      "Django": {
        "match_percentage": 0,
        "explanation": "No experience with Django is mentioned in the candidate's evaluation."
      },
      "RESTful_API_design_and_implementation": {
        "match_percentage": 30,
        "explanation": "Candidate's experience with Flask suggests some backend API experience, but it's not explicitly stated as RESTful API design and implementation. More information is needed."
      },
      "cloud_platforms_(AWS_or_Azure)": {
        "match_percentage": 20,
        "explanation": "Candidate has experience with Firebase, but not AWS or Azure as specified in the job description."
      },
      "CI/CD_pipelines_and_DevOps_practices": {
        "match_percentage": 0,
        "explanation": "No experience with CI/CD pipelines or DevOps practices is mentioned in the candidate's evaluation."
      },
      "experience_level": {
        "match_percentage": 40,
        "explanation": "Candidate's experience seems to align with a mid-level developer rather than a senior one, based on the lack of experience in essential technologies like React.js and Django."
      }
    },
    "preferred_requirements_match": {
      "TypeScript": {
        "match_percentage": 0,
        "explanation": "No experience with TypeScript is mentioned."
      },
      "Node.js": {
        "match_percentage": 0,
        "explanation": "No experience with Node.js is mentioned."
      },
      "Docker": {
        "match_percentage": 40,
        "explanation": "Candidate mentions Docker in skills but lacks specific application examples. Further evidence is needed."
      },
      "database_design_and_optimization": {
        "match_percentage": 50,
        "explanation": "Candidate has experience with MongoDB and MySQL, but the level of proficiency and experience with database design and optimization needs clarification."
      },
      "microservices_architecture": {
        "match_percentage": 0,
        "explanation": "No experience with microservices architecture is mentioned."
      },
      "mentoring_junior_developers": {
        "match_percentage": 60,
        "explanation": "Candidate's experience leading a team suggests potential for mentoring, but explicit experience mentoring junior developers is not mentioned."
      }
    },
    "overall_match_percentage": 35,
    "explanation": "The candidate possesses strong skills in several areas, particularly machine learning and data science. However, they lack crucial experience in several essential technologies required for the Senior Full Stack Developer role (React.js, Django, RESTful APIs, AWS/Azure, CI/CD). While their overall technical skills are good, their experience doesn't fully align with the senior-level expectations of this position.  Significant growth in specific areas is needed."
  }
}
```

## Technical Interview Question Generator
```json
{
  "questions": [
    {
      "question": "In your Infosys internship, you led a team in building an NLP system for automating ticket classification and responses. Can you walk me through the architecture of your system, highlighting the key components and the technologies you used?  Discuss any challenges you faced and how you overcame them.  What metrics did you use to evaluate the system's performance, and what were the results?",
      "assessment": "Assess depth of knowledge in NLP, team leadership, system design, and problem-solving.  Evaluate understanding of NLP pipelines, model selection, evaluation metrics, and deployment challenges.",
      "good_answer": "A good answer would detail the system architecture (e.g., data preprocessing, feature extraction, model training, deployment), specific technologies used (e.g., libraries, frameworks), challenges encountered (e.g., data imbalance, model accuracy), solutions implemented, evaluation metrics (e.g., precision, recall, F1-score), and quantifiable results (e.g., percentage improvement in efficiency)."
    },
    {
      "question": "Your research involved using LANDSAT data and machine learning to monitor rangeland vegetation.  Describe a specific machine learning model you used and justify your choice.  How did you handle the complexities of working with remote sensing data, such as noise and variations in image quality?",
      "assessment": "Assess depth of knowledge in machine learning, remote sensing, and data preprocessing.  Evaluate ability to justify model selection and handle real-world data challenges.",
      "good_answer": "A good answer would describe a specific model (e.g., Random Forest, CNN), justify its selection based on the data characteristics and problem requirements, and detail how they addressed challenges like data cleaning, feature engineering, and handling noisy or missing data.  Mentioning specific techniques used for image preprocessing would be beneficial."
    },
    {
      "question": "Your DeepFake detection project involved a pipeline of MTCNN, ResNet, and LSTM.  Explain the role of each component in the pipeline and why you chose this specific combination.  How did you improve the model's accuracy by 15%, and what were the limitations of your approach?",
      "assessment": "Assess depth of knowledge in computer vision, deep learning, and model improvement techniques.  Evaluate understanding of model architectures and limitations.",
      "good_answer": "A good answer would clearly explain the function of each component (MTCNN for face detection, ResNet for feature extraction, LSTM for temporal analysis), justify the choice of these models, detail the techniques used to improve accuracy (e.g., data augmentation, hyperparameter tuning, model ensembling), and discuss the limitations of the approach (e.g., computational cost, generalization to unseen data)."
    },
    {
      "question": "Your NurtureSync project utilized both Flutter and Streamlit.  Describe the architectural decisions you made in choosing these technologies for the mobile and web applications, respectively.  How did you handle data synchronization between the mobile and web platforms, and what challenges did you encounter in integrating Firebase and MongoDB?",
      "assessment": "Assess depth of knowledge in Flutter, Streamlit, Firebase, and MongoDB.  Evaluate understanding of architectural design, data synchronization, and integration challenges.",
      "good_answer": "A good answer would explain the rationale behind choosing Flutter and Streamlit, describe the architecture (e.g., client-server model, API design), detail the data synchronization mechanism (e.g., real-time database updates, API calls), and discuss challenges encountered during integration (e.g., data consistency, error handling, scalability)."
    },
    {
      "question": "Design a system for real-time DeepFake video detection as a Chrome extension. Consider scalability, latency requirements, and security concerns.  What are the key components of your system, and how would you address potential challenges such as resource constraints and privacy issues?",
      "assessment": "Assess system design skills, focusing on real-time processing, scalability, security, and resource constraints.  Evaluate ability to design a system considering various constraints and potential challenges.",
      "good_answer": "A good answer would outline a system architecture that addresses real-time processing needs (e.g., efficient model inference, optimized data flow), scalability (e.g., cloud-based processing, distributed architecture), security (e.g., data encryption, secure communication channels), and privacy (e.g., anonymization techniques, minimal data collection).  The answer should also discuss strategies for handling resource constraints and potential errors."
    },
    {
      "question": "You mentioned using Selenium for automated testing.  Describe a scenario where you had to debug a complex issue in a web application using Selenium.  What debugging techniques did you employ, and how did you approach isolating and resolving the problem?",
      "assessment": "Assess problem-solving skills and practical experience with Selenium.  Evaluate ability to debug complex issues and effectively utilize debugging techniques.",
      "good_answer": "A good answer would describe a specific scenario, detail the debugging steps taken (e.g., logging, breakpoints, network inspection), and explain how they systematically isolated and resolved the problem.  Mentioning specific Selenium commands and techniques would be beneficial."
    },
    {
      "question": "Given a large dataset of text messages, how would you build a system to classify them into different categories (e.g., spam, promotional, personal)?  Describe your approach, including data preprocessing, feature engineering, model selection, and evaluation metrics.",
      "assessment": "Assess problem-solving skills and knowledge of NLP techniques.  Evaluate ability to design a complete NLP pipeline for text classification.",
      "good_answer": "A good answer would outline a complete NLP pipeline, including data cleaning (e.g., removing punctuation, handling special characters), feature engineering (e.g., TF-IDF, word embeddings), model selection (e.g., Naive Bayes, SVM, deep learning models), training, and evaluation using appropriate metrics (e.g., accuracy, precision, recall, F1-score)."
    }
  ]
}
```

## Job Requirements Specialist
```json
{
  "assessment": {
    "essentialRequirements": [
      {
        "requirement": "5+ years of experience in software development",
        "matchPercentage": 90,
        "explanation": "While the candidate's resume doesn't explicitly state years of experience, the breadth and depth of projects (NurtureSync, DeepFake detection, internships) strongly suggest 5+ years of experience.  Further clarification on the timeline of these projects would solidify this assessment.",
        "matchType": "Indirect (Inferred from project portfolio)"
      },
      {
        "requirement": "Strong proficiency in Python and JavaScript",
        "matchPercentage": 75,
        "explanation": "Python proficiency is high (rating 8), demonstrating strong practical application. JavaScript proficiency is moderate (rating 6), requiring further demonstration of advanced skills.  The candidate's experience with Python is more extensive and demonstrably stronger.",
        "matchType": "Direct"
      },
      {
        "requirement": "Experience with React.js and Django",
        "matchPercentage": 30,
        "explanation": "The candidate's skillset doesn't directly include React.js or Django.  While Flask and Streamlit show backend and frontend experience, they are not direct equivalents.  This represents a significant gap.",
        "matchType": "No Match - Critical Gap"
      },
      {
        "requirement": "Knowledge of RESTful API design and implementation",
        "matchPercentage": 70,
        "explanation": "The use of Firebase in NurtureSync suggests familiarity with API design and implementation, but the extent of this knowledge needs further clarification.  The candidate's projects demonstrate some understanding, but deeper experience is needed.",
        "matchType": "Indirect (Inferred from project usage)"
      },
      {
        "requirement": "Experience with cloud platforms (AWS or Azure)",
        "matchPercentage": 0,
        "explanation": "No mention or evidence of experience with AWS or Azure.",
        "matchType": "No Match - Critical Gap"
      },
      {
        "requirement": "Understanding of CI/CD pipelines and DevOps practices",
        "matchPercentage": 20,
        "explanation": "The candidate's skillset doesn't directly mention CI/CD or DevOps practices.  While Git usage is strong, this is only a small part of the broader CI/CD and DevOps landscape. This is a significant gap.",
        "matchType": "No Match - Critical Gap"
      }
    ],
    "preferredRequirements": [
      {
        "requirement": "Experience with TypeScript and Node.js",
        "matchPercentage": 0,
        "explanation": "No mention or evidence of experience with TypeScript or Node.js.",
        "matchType": "No Match"
      },
      {
        "requirement": "Knowledge of container technologies (Docker, Kubernetes)",
        "matchPercentage": 10,
        "explanation": "Docker is mentioned in the resume but lacks project evidence.  Kubernetes is not mentioned.",
        "matchType": "No Match"
      },
      {
        "requirement": "Experience with database design and optimization",
        "matchPercentage": 40,
        "explanation": "The candidate has used MongoDB and mentions MySQL, but lacks demonstrable experience in database design and optimization.  Further evidence is needed.",
        "matchType": "Partial Match"
      },
      {
        "requirement": "Familiarity with microservices architecture",
        "matchPercentage": 0,
        "explanation": "No mention or evidence of familiarity with microservices architecture.",
        "matchType": "No Match"
      },
      {
        "requirement": "Experience mentoring junior developers",
        "matchPercentage": 0,
        "explanation": "No evidence of mentoring experience.",
        "matchType": "No Match"
      }
    ],
    "experienceLevelAlignment": {
      "matchPercentage": 70,
      "explanation": "The candidate's project portfolio suggests a strong foundation, but lacks the specific experience with technologies like React.js, Django, AWS/Azure, and CI/CD that are crucial for a Senior Full Stack Developer role.  More experience is needed to fully align with the seniority level."
    },
    "culturalFitIndicators": {
      "matchPercentage": 80,
      "explanation": "The candidate's active GitHub profile, diverse project portfolio, and willingness to learn suggest a good cultural fit.  Their practical application of skills and focus on results are positive indicators.  Further assessment of teamwork and communication skills would be beneficial."
    },
    "growthPotential": {
      "matchPercentage": 90,
      "explanation": "The candidate demonstrates a strong learning ability and actively seeks new challenges.  Their identified growth areas are addressable through focused learning and experience.  The candidate has the potential to quickly acquire the missing skills."
    },
    "overallMatchPercentage": 45,
    "explanation": "The overall match percentage is weighted heavily towards essential requirements.  Significant gaps exist in crucial areas like React.js, Django, cloud platforms, CI/CD, and DevOps. While the candidate possesses a strong foundation and learning agility, these gaps need to be addressed before they can be considered a strong match for this senior-level role."
  }
}
```

## Evaluation Pipeline Manager
```json
{
  "evaluationPipeline": {
    "stage1": {
      "name": "Basic Eligibility Check",
      "maxPoints": 20,
      "score": 15,
      "justification": "The candidate meets the basic education requirement (Bachelor of Engineering in AI).  However, the years of experience are inferred from projects, not explicitly stated, resulting in a deduction of 5 points.  Certifications are partially verified, leading to further point reduction. Location verification is not applicable in this case.",
      "keyFactors": {
        "education": "Met",
        "experience": "Inferred (partially met)",
        "certifications": "Partially verified"
      }
    },
    "stage2": {
      "name": "Skill Match Evaluation",
      "maxPoints": 40,
      "score": 28,
      "justification": "The candidate demonstrates strong proficiency in Python and several relevant frameworks and tools.  However, significant gaps exist in essential requirements like React.js, Django, and cloud platforms.  Essential skills are weighted more heavily, resulting in a moderate score.",
      "keyFactors": {
        "matchedSkills": ["Python", "Flutter", "Flask", "Streamlit", "Selenium", "Firebase", "MongoDB", "ML", "NLP", "DL", "Testing"],
        "missingSkills": ["React.js", "Django", "AWS/Azure", "CI/CD", "DevOps"],
        "bonusSkills": []
      },
      "skillWeights": {
        "essential": 0.7,
        "preferred": 0.3
      }
    },
    "stage3": {
      "name": "Domain-Specific Experience Check",
      "maxPoints": 15,
      "score": 10,
      "justification": "The candidate's projects and research experience demonstrate some relevance to the AI/ML domain.  However, the depth of experience in specific domain areas needs further clarification.  The publication record adds to the score.",
      "keyFactors": {
        "projects": "Relevant projects in AI/ML",
        "research": "Demonstrates domain knowledge",
        "publications": "Adds to domain expertise"
      }
    },
    "stage4": {
      "name": "Role Fit and Achievements Alignment",
      "maxPoints": 25,
      "score": 22,
      "justification": "The candidate's achievements, particularly the Health Hackathon finalist position and research contributions, demonstrate impact and initiative.  Leadership experience in the Infosys internship is a positive factor.  However, direct alignment with specific job responsibilities needs further assessment.",
      "keyFactors": {
        "achievements": "Demonstrates impact and initiative",
        "leadership": "Present in Infosys internship",
        "alignment": "Partially aligned with job responsibilities"
      }
    },
    "total": {
      "maxScore": 100,
      "score": 75,
      "recommendation": "Proceed to interview",
      "additionalNotes": "While the candidate shows strong potential and a solid foundation, the significant gaps in essential skills identified in Stage 2 should be addressed during the interview process.  Further investigation into the timeline and depth of projects is recommended to better assess experience level."
    }
  }
}
```

## Technical Interview Question Generator
```json
{
  "questions": [
    {
      "question": "Your NurtureSync project uses Flask, Streamlit, MongoDB, and Firebase.  Describe the architecture of your application, focusing on how these technologies interact.  Explain your choice of technologies for each component and any challenges you faced during integration.",
      "assessment": "Assesses understanding of application architecture, technology choices, and integration challenges.  Evaluates problem-solving skills and ability to explain technical decisions.",
      "difficulty": "Intermediate",
      "goodAnswer": "A good answer would detail the interaction between the frontend (Streamlit), backend (Flask), database (MongoDB), and backend services (Firebase).  It should justify the choice of each technology based on its strengths and address challenges encountered during development, such as data synchronization between the mobile and web apps or handling real-time updates."
    },
    {
      "question": "In your DeepFake detection project, you used MTCNN, ResNet, and LSTM.  Explain the role of each component in your pipeline and the rationale behind your choice of these specific models.  How did you address the challenges of real-time video processing?",
      "assessment": "Assesses understanding of Deep Learning models and their application in Computer Vision.  Evaluates problem-solving skills and ability to explain technical choices.",
      "difficulty": "Advanced",
      "goodAnswer": "A good answer would clearly explain the function of each component (MTCNN for face detection, ResNet for feature extraction, LSTM for temporal analysis) and justify the selection based on their strengths and suitability for DeepFake detection.  It should also discuss challenges related to real-time processing, such as computational cost and optimization techniques used to improve performance."
    },
    {
      "question": "Your resume mentions experience with Selenium for automated testing. Describe a complex testing scenario you encountered at Baavlibuch and how you approached automating the tests using Selenium. What challenges did you face, and how did you overcome them?",
      "assessment": "Assesses practical experience with Selenium and problem-solving skills in a testing context.  Evaluates ability to handle complex scenarios and explain technical solutions.",
      "difficulty": "Intermediate",
      "goodAnswer": "A good answer would describe a specific, complex testing scenario, outlining the steps involved in automating the tests using Selenium.  It should highlight challenges encountered, such as dealing with dynamic web elements or handling asynchronous operations, and explain the solutions implemented to overcome these challenges."
    },
    {
      "question": "Design a system for real-time monitoring of rangeland vegetation using LANDSAT data, similar to your research project.  Consider scalability, data processing, and visualization aspects.  What technologies would you choose and why?",
      "assessment": "Assesses system design skills, understanding of data processing pipelines, and ability to choose appropriate technologies for a real-world problem.  Evaluates ability to think about scalability and performance.",
      "difficulty": "Advanced",
      "goodAnswer": "A good answer would outline a system architecture for real-time monitoring, including data ingestion, preprocessing, analysis, and visualization components.  It should justify the choice of technologies (e.g., cloud platforms for storage and processing, specific libraries for data analysis and visualization) based on their suitability for the task and scalability requirements."
    },
    {
      "question": "You have extensive experience with Python and Flask.  Let's say you need to build a RESTful API for NurtureSync using Django instead of Flask.  What are the key differences you would encounter, and what changes would you need to make to your existing codebase?",
      "assessment": "Assesses adaptability and understanding of different web frameworks.  Evaluates ability to transfer knowledge and adapt to new technologies.",
      "difficulty": "Intermediate",
      "goodAnswer": "A good answer would identify key differences between Flask and Django (e.g., ORM, project structure, templating engines) and explain how these differences would impact the development process.  It should outline the necessary changes to the codebase to migrate from Flask to Django while maintaining functionality."
    },
    {
      "question": "Imagine you need to deploy your NurtureSync application to AWS.  Describe the steps involved in setting up the infrastructure, deploying the application, and ensuring scalability and reliability.  What services would you use and why?",
      "assessment": "Assesses understanding of cloud deployment and scalability.  Evaluates ability to apply knowledge to a new context and address practical challenges.",
      "difficulty": "Advanced",
      "goodAnswer": "A good answer would outline the steps involved in deploying NurtureSync to AWS, including choosing appropriate services (e.g., EC2, S3, RDS, Lambda), configuring the infrastructure, and implementing strategies for scalability and reliability (e.g., load balancing, auto-scaling).  It should also address security considerations."
    },
    {
      "question": "You mentioned working with Linux in your Baavlibuch internship.  Let's say you need to implement a CI/CD pipeline for your DeepFake detection project using Docker and Jenkins.  Describe the steps involved in setting up the pipeline and automating the build, testing, and deployment processes.",
      "assessment": "Assesses understanding of CI/CD pipelines and Docker.  Evaluates ability to apply knowledge to a new context and address practical challenges.",
      "difficulty": "Advanced",
      "goodAnswer": "A good answer would outline the steps involved in setting up a CI/CD pipeline using Docker and Jenkins, including creating Docker images, configuring Jenkins jobs, automating the build and testing processes, and deploying the application to a target environment.  It should also address version control and error handling."
    }
  ]
}
```